# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

J. Braspenning  ->  J. Braspenning  |  ['J. Braspenning']
A. Pillepich  ->  A. Pillepich  |  ['A. Pillepich']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']


Arxiv has 77 new papers today
          2 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/2 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2511.09649
extracting tarball to tmp_2511.09649... done.


Retrieving document from  https://arxiv.org/e-print/2511.09770
extracting tarball to tmp_2511.09770...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.09649-b31b1b.svg)](https://arxiv.org/abs/2511.09649) | **The origin of scatter in the X-ray luminosity - halo mass relation of galaxy clusters**  |
|| <mark>J. Braspenning</mark>, J. Schaye, <mark>A. Pillepich</mark>, D. Nelson |
|*Appeared on*| *2025-11-14*|
|*Comments*| *19 pages, 11 figures (including appendix). Submitted to MNRAS*|
|**Abstract**|            Galaxy groups and clusters are excellent probes of large-scale structure and are shaped by some of the most energetic physical processes in the Universe. They follow a tight scaling relation of X-ray luminosity with halo mass. However, predicting the dependence of the scatter in this relation on mass and redshift is challenging, due to the statistical requirement of large simulation volumes. Using the large volume cosmological hydrodynamical simulations for galaxy cluster physics FLAMINGO and TNG300+TNG-Cluster, we fit this relation and its scatter, focusing on $M_{\rm 500c}>10^{13}~\mathrm{M_\odot}$ and $z \leq 2$. We find qualitatively similar, but quantitatively different results for the two models. For the first time, we study ways to reduce the scatter using properties beyond X-ray luminosity, namely six ICM, six galaxy, and eleven dark matter halo properties. For both FLAMINGO and TNG300+TNG-Cluster, the gas fraction and thermal Sunyaev-Zel'dovich (SZ) signal correlate strongest with X-ray scatter, reducing it by over 50% when accounting for their partial correlations. Galaxy and halo properties correlate weakly with X-ray scatter, typically reducing it by 10-20%. Our results are qualitatively robust across different FLAMINGO feedback variations, though the correlations weaken for stronger feedback and with increasing redshift. Differences between FLAMINGO and TNG300+TNG-Cluster are only apparent at the high-mass end - where e.g. the galaxy stellar age correlates strongly for FLAMINGO, but not for TNG300+TNG-Cluster - confirming robustness across physics implementations. We provide fitting formulas for the scatter and its corrections, for direct application to cosmological analyses and observational data.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.09770-b31b1b.svg)](https://arxiv.org/abs/2511.09770) | **Searching for Long-Period Radio Transients in ASKAP EMU Data with 10-Second Imaging**  |
|| Y. W. J. Lee, et al. -- incl., <mark>Y. Wang</mark> |
|*Appeared on*| *2025-11-14*|
|*Comments*| *19 pages, 14 figures, 5 tables*|
|**Abstract**|            Long-period radio transients (LPTs) are a recently identified phenomenon that challenge our current understanding of compact objects and coherent radio emission mechanisms. These objects emit radio pulses similar to those of pulsars, but at much longer periods -- on the order of minutes to hours. With duty cycles of only a few percent, individual pulses have been observed to last between 10 and 1000 seconds. This places LPTs in a timescale gap between the two main techniques used in transient radio searches: time-series analysis at millisecond to second timescales, and image-plane searches sensitive to variability on the scale of days. As a result, LPTs remained undetected until recently, and only a handful are currently known. To increase the sample of known LPTs, we conducted a dedicated search using 200 hours of archival data from the ASKAP Evolutionary Map of the Universe survey, covering 750 deg$^2$ of sky at the shortest possible imaging time step of 10-seconds. This represents the first large-scale search using ASKAP data at second-scale resolution. Although no LPTs were detected, we identified flares from six stars, at least one had never been detected in the radio regime before. We placed a lower limit on the transient surface density of $2.21\times10^{-6}$ deg$^{-2}$ at a 10-second timescale, with a sensitivity of 16.9 mJy. Our findings evaluate the feasibility of detecting radio transients using 10-second imaging with ASKAP and provide insights into improving detection pipelines and observation strategies for LPTs.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2511.09649/./Figures/scatter_fits_flamingo_fid_crop.png', 'tmp_2511.09649/./Figures/scatter_fits_tng_crop.png', 'tmp_2511.09649/./Figures/gas_spearman_13.52.png', 'tmp_2511.09649/./Figures/gas_spearman_14.50.png']
copying  tmp_2511.09649/./Figures/scatter_fits_flamingo_fid_crop.png to _build/html/
copying  tmp_2511.09649/./Figures/scatter_fits_tng_crop.png to _build/html/
copying  tmp_2511.09649/./Figures/gas_spearman_13.52.png to _build/html/
copying  tmp_2511.09649/./Figures/gas_spearman_14.50.png to _build/html/
exported in  _build/html/2511.09649.md
    + _build/html/tmp_2511.09649/./Figures/scatter_fits_flamingo_fid_crop.png
    + _build/html/tmp_2511.09649/./Figures/scatter_fits_tng_crop.png
    + _build/html/tmp_2511.09649/./Figures/gas_spearman_13.52.png
    + _build/html/tmp_2511.09649/./Figures/gas_spearman_14.50.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# The origin of scatter in the X-ray luminosity -- halo mass relation of galaxy clusters

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2511.09649-b31b1b.svg)](https://arxiv.org/abs/2511.09649)<mark>Appeared on: 2025-11-14</mark> -  _19 pages, 11 figures (including appendix). Submitted to MNRAS_

</div>
<div id="authors">

<mark>J. Braspenning</mark>, J. Schaye, <mark>A. Pillepich</mark>, D. Nelson

</div>
<div id="abstract">

**Abstract:** Galaxy groups and clusters are excellent probes of large-scale structure and are shaped by some of the most energetic physical processes in the Universe. They follow a tight scaling relation of X-ray luminosity with halo mass. However, predicting the dependence of the scatter in this relation on mass and redshift is challenging, due to the statistical requirement of large simulation volumes. Using the large volume cosmological hydrodynamical simulations for galaxy cluster physics FLAMINGO and TNG300+TNG-Cluster, we fit this relation and its scatter, focusing on $M_{\rm 500c}>10^{13} \mathrm{M_\odot}$ and $z \leq 2$ . We find qualitatively similar, but quantitatively different results for the two models. For the first time, we study ways to reduce the scatter using properties beyond X-ray luminosity, namely six ICM, six galaxy, and eleven dark matter halo properties. For both FLAMINGO and TNG300+TNG-Cluster, the gas fraction and thermal Sunyaev-Zel \' dovich (SZ) signal correlate strongest with X-ray scatter, reducing it by over 50 \% when accounting for their partial correlations. Galaxy and halo properties correlate weakly with X-ray scatter, typically reducing it by 10-20 \% . Our results are qualitatively robust across different FLAMINGO feedback variations, though the correlations weaken for stronger feedback and with increasing redshift.Differences between FLAMINGO and TNG300+TNG-Cluster are only apparent at the high-mass end -- where e.g. the galaxy stellar age correlates strongly for FLAMINGO, but not for TNG300+TNG-Cluster -- confirming robustness across physics implementations. We provide fitting formulas for the scatter and its corrections, for direct application to cosmological analyses and observational data.

</div>

<div id="div_fig1">

<img src="tmp_2511.09649/./Figures/scatter_fits_flamingo_fid_crop.png" alt="Fig7.1" width="50%"/><img src="tmp_2511.09649/./Figures/scatter_fits_tng_crop.png" alt="Fig7.2" width="50%"/>

**Figure 7. -** Redshift evolution of the scatter in the $L_{\rm X}$ -- $M_{\rm 500c}$ relation for FLAMINGO $\mathrm{L2p8}$(top) and TNG300+TNG-Cluster (bottom). The left panels show the scatter as a function of mass from $z=0$ to $z=2$(solid curves), with shaded areas depicting the $1\sigma$ bootstrap error in each bin. The dotted curves show the scatter for the core-excised luminosity $L_{\rm X, ce}$, which only differs significantly from the scatter in the total luminosity either below $10^{14} \mathrm{M_{\odot}}$(FLAMINGO) or above that mass (TNG300+TNG-Cluster). A best-fit broken power law for each redshift is shown as dashed lines of the same colour, with arrows indicating the break point. The middle four-tile panels show the redshift evolution of the four parameters of the broken power law (Eq. \ref{eq:broken_powerlaw_scatter}), and the best fit quadratic function (Eq. \ref{eq:redshift_evolution_broken_powerlaw}) to that evolution. The right most panels show the difference between the true scatter in each bin, and the prediction from the broken power law using the parameters at each redshift from the best quadratic fit shown in the middle panels. The quadratic fit to the redshift evolution, combined with the broken power law, allows an accurate prediction, within $0.01 \mathrm{dex}$(FLAMINGO) and $0.05 \mathrm{dex}$(TNG300+TNG-Cluster), across a wide range of redshift and mass. (*fig:scatter_redshift_FLAMINGO_TNG*)

</div>
<div id="div_fig2">

<img src="tmp_2511.09649/./Figures/gas_spearman_13.52.png" alt="Fig8" width="100%"/>

**Figure 8. -** The logarithmic deviation from the $L_{\rm X}$ -- $M_{\rm 500c}$ scaling relation (in dex; Eq. \ref{eq:log_deviation}) against the logarithmic deviation in a second cluster property (selected from Table \ref{tab:all_properties}) for FLAMINGO L2p8 at $z=0$ at $M_{\rm 500c} = 10^{13.5}   \mathrm{M_\odot}$(30,000 haloes). Lighter colours indicate a larger number of haloes. The red error bars (barely visible) are 16th-84th percentile bootstrap resamplings of the median in ten logarithmic bins of the second property. The dashed red line is the best linear fit for those error bars. The spearman rank correlation is indicated in the top left of each panel. Some of the properties show a clear correlation with the deviation from the X-ray luminosity scaling relation, but the level of correlation and the scatter differ strongly between properties. (*fig:second_property_137*)

</div>
<div id="div_fig3">

<img src="tmp_2511.09649/./Figures/gas_spearman_14.50.png" alt="Fig9" width="100%"/>

**Figure 9. -** As Fig. \ref{fig:second_property_137}, for FLAMINGO L2p8 at $z=0$, but for the higher-mass clusters with $M_{\rm 500c} = 10^{14.5}   \mathrm{M_\odot}$(3771 haloes). (*fig:second_property_147*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2511.09649"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

121  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

6  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
